In [ ]:
import os
from pprint import pprint
from dotenv import load_dotenv
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from neo4j.exceptions import ConfigurationError
from langchain_huggingface import HuggingFaceEmbeddings
from src.grag import create_vector_cypher_retriever_tool

load_dotenv(".env")

In [ ]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]
DATABASE = os.environ["DATABASE_SMALL"]

neo4j_config = {
    "DATABASE_NAME": DATABASE,
    "ARTICLE_VECTOR_INDEX_NAME": os.environ["ARTICLE_VECTOR_INDEX_NAME"],
    "ARTICLE_FULLTEXT_INDEX_NAME": os.environ["ARTICLE_FULLTEXT_INDEX_NAME"],
    "DEFINITION_VECTOR_INDEX_NAME": os.environ["DEFINITION_VECTOR_INDEX_NAME"],
    "DEFINITION_FULLTEXT_INDEX_NAME": os.environ["DEFINITION_FULLTEXT_INDEX_NAME"],
}

# Ngga jadi pakai ini karena Text2Cypher mintannya harus pakai Neo4jGraph
# neo4j_driver = GraphDatabase.driver(uri=URI, auth=(USERNAME, PASSWORD))

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

neo4j_driver = neo4j_graph._driver  # Ambil driver nya kaya gini
embedder_model = HuggingFaceEmbeddings(model_name=os.environ["EMBEDDING_MODEL"])

In [ ]:
vector_cypher_retriever = create_vector_cypher_retriever_tool(
    embedder_model=embedder_model,
    neo4j_driver=neo4j_driver,
    neo4j_config=neo4j_config,
    top_k_initial_article=5,
    total_article_limit=10,
    total_definition_limit=10
)

In [ ]:
query = "Apa itu penyelenggara sistem elektronik?"

In [ ]:
result = vector_cypher_retriever.invoke({"query": query})
print(result)

In [ ]:
# Agar hasilnya dalam ToolMessage, harus invoke menggunakan schema ToolCall
# https://python.langchain.com/docs/how_to/tool_artifacts/#invoking-the-tool-with-toolcall

result = vector_cypher_retriever.invoke(
    {
        "name": "vector_cypher_retriever",
        "args": {"query": query},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

result.model_dump()

In [ ]:
len(result.content)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

checkpointer = MemorySaver()
config = {"configurable": {"thread_id": "1"}}

react_agent = create_react_agent(model=llm, tools=[vector_cypher_retriever], checkpointer=checkpointer, version="v2")
print(react_agent.get_graph().draw_ascii())

In [ ]:
query = "Apa itu penyelenggara sistem elektronik?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Wajib gunakan tool vector_cypher_retriever")},
    config=config
)

result["messages"]

In [ ]:
query = "Bagaimana dengan definisi data pribadi?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query)},
    config=config
)

result["messages"]

In [ ]:
query = "Tadi definisi penyelenggara sistem elektronik apa? Aku lupa"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Ngga perlu pakai tool call lagi ya")},
    config=config
)

result["messages"]

In [ ]:
query = "Apa hubungan penyelenggara sistem elektronik dan data pribadi?"
result = react_agent.invoke(
    {"messages": HumanMessage(content=query + " Wajib gunakan tool vector_cypher_retriever")},
    config=config
)

result["messages"]

In [ ]:
result["messages"][2].model_dump()

In [ ]:
from langchain_core.messages import AIMessage

ai_results = []

for message in result["messages"]:
    if isinstance(message, HumanMessage):
        query = message.content
    if isinstance(message, AIMessage):
        # ai_results.append(message)
        if message.tool_calls:
            ai_results.append({
                "type": "tool_call",
                "id": message.id,
                "query": query,
                "usage": message.usage_metadata
            })
        else:
            ai_results.append({
                "type": "model_response",
                "id": message.id,
                "query": query,
                "response": message.content,
                "usage": message.usage_metadata
            })
                

for ai_result in ai_results:
    pprint(ai_result)
    print()

In [ ]:
test = llm.invoke("Halo apa yang sedang kamu lakukan? Jawab singkat saja ya")
print(test.content)
print(test.usage_metadata)

In [ ]:
result['messages'][2].model_dump()['artifact']

In [ ]:
test = llm.invoke(f"Jelaskan data ini: {result['messages'][2].model_dump()['artifact']}")
print(test.content)
print(test.usage_metadata)